# Media Cost Campaign

the dataset come from kaggle' competition : 
https://www.kaggle.com/competitions/playground-series-s3e11/data

the main goal is to determine the cost with the best accuracy


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Feature scaling if necessary
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# To evaluate the model
from sklearn.metrics import make_scorer,mean_squared_log_error

# Linear regression
from sklearn.linear_model import LinearRegression 

# Ridge regression
from sklearn.linear_model import Ridge

# Lasso regression
from sklearn.linear_model import Lasso

# Random Forrest Regressor
from sklearn.ensemble import RandomForestRegressor

# xgboost model and feature importance
from xgboost import XGBRegressor,plot_importance

# lightgbm model 
from lightgbm import LGBMRegressor

# catboost Regressor model
from catboost import CatBoostRegressor

# Extra Trees Regressor model
from sklearn.ensemble import ExtraTreesRegressor

# Tuning hyperparameters
from sklearn.model_selection import GridSearchCV

RANDOM_STATE = 17

In [4]:


# Playground dataset train & test
P_train_df = pd.read_csv('train.csv')
P_test_df = pd.read_csv('test.csv')

# Original dataset train & test
O_test_df = pd.read_csv('Original_test_dataset.csv')
O_train_df = pd.read_csv('Original_train_dataset.csv')



FileNotFoundError: [Errno 2] No such file or directory: 'Original_train_dataset.csvcsv'